# **Data Handling :**
In the following code blocks, we have read the h5 dataset (TVSum) and splitted it into train and test set.

In [ ]:
# Importing essential libraries
import h5py
import numpy as np
import tensorflow.keras as Keras
from sklearn.model_selection import train_test_split

In [ ]:
# Mount your drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Defining a Data Generator class to handle features of h5 file
class DataGenerator(Keras.utils.Sequence):

    def __init__(self,dataset,batch_size=5,shuffle=False):
      ''' Initialize the data'''

        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
      ''' Returns length of the dataset'''

        return int(np.floor(len(self.dataset)/self.batch_size))

    def __getitem__(self,index):
      ''' Returns index, features and labels of the dataset'''

        indexes = self.indices[index * self.batch_size : (index+1) * self.batch_size]
        feature, label = self.__data_generation(indexes)

        return feature, label

    def __data_generation(self,indexes):
      '''Returns features and labels'''

        feature = np.empty((self.batch_size,320,1024))
        label = np.empty((self.batch_size,320,1))

        for i in range(len(indexes)):
            feature[i,] = np.array(self.dataset[indexes[i]][0])
            label[i,] = np.array(self.dataset[indexes[i]][1]).reshape(-1,1)

        return feature,label

    def on_epoch_end(self):
      '''Shuffles indices for epochs'''

        self.indices = np.arange(len(self.dataset))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

In [ ]:
# Defining a class to read the h5 dataset
class DatasetMaker(object):

    def __init__(self,data_path):
      '''Read file from defined path'''

        self.data_file = h5py.File(data_path)

    def __len__(self):
      '''Returns length of the file'''

        return len(self.data_file)

    def __getitem__(self,index):
      '''Returns feature, label and index of varoius keys'''

        index += 1
        video = self.data_file['video_'+str(index)]
        feature = np.array(video['feature'][:])
        label = np.array(video['label'][:])

        return feature,label,index

# Defining a function to read and split the dataset into train and test
def get_loader(path, batch_size=5):
  '''Takes file path as argument and returns train and test set'''

    dataset = DatasetMaker(path)
    train_dataset, test_dataset = train_test_split(dataset, test_size = 0.2)
    train_loader = DataGenerator(train_dataset)

    return train_loader, test_dataset

In [ ]:
# Getting train and test set
train_loader, test_dataset=get_loader('/content/drive/MyDrive/suvidha internship/fcsn_tvsum.h5')

In [ ]:
# Getting an overview of train set
train_loader.__getitem__(1)

(array([[[0.35250884, 0.35687858, 0.37622839, ..., 0.4116306 ,
          0.37593332, 0.41581485],
         [0.33224654, 0.381078  , 0.34691849, ..., 0.41855153,
          0.40180525, 0.39628667],
         [0.35970178, 0.37046221, 0.35555354, ..., 0.41693273,
          0.40011674, 0.42686591],
         ...,
         [0.37487707, 0.3455686 , 0.3770965 , ..., 0.41539136,
          0.39820242, 0.36128262],
         [0.37143588, 0.35041133, 0.38537401, ..., 0.42188835,
          0.41068125, 0.3635914 ],
         [0.37933877, 0.35179704, 0.38787192, ..., 0.41555437,
          0.40098241, 0.35158965]],
 
        [[0.34310278, 0.32379588, 0.3629739 , ..., 0.41467142,
          0.4268606 , 0.40030608],
         [0.3324433 , 0.34779271, 0.33534241, ..., 0.41157749,
          0.38300651, 0.40324458],
         [0.34330019, 0.34221551, 0.31614497, ..., 0.41342187,
          0.38614759, 0.41145793],
         ...,
         [0.36588389, 0.3762069 , 0.33913556, ..., 0.43120918,
          0.3935608 , 0.

In [ ]:
# Getting an overview of test set
test_dataset[1]

(array([[0.319943  , 0.37948456, 0.32990432, ..., 0.43006426, 0.38723537,
         0.43616933],
        [0.32591122, 0.3689806 , 0.34779176, ..., 0.4292725 , 0.3834775 ,
         0.4353265 ],
        [0.32701132, 0.37345025, 0.344078  , ..., 0.43138295, 0.38321468,
         0.43406087],
        ...,
        [0.3387942 , 0.36302942, 0.3892704 , ..., 0.41417256, 0.38435096,
         0.39531755],
        [0.35451224, 0.35858825, 0.35803077, ..., 0.44103327, 0.41493303,
         0.4012603 ],
        [0.3536804 , 0.3568709 , 0.35599893, ..., 0.44360656, 0.41518053,
         0.3995843 ]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 0.

# **Model Training and Testing :**
In this section, we have defined various functions to handle the algorithm in order and trained the model and tested it over the test set.

In [ ]:
import numpy as np

def knapsack(s, w, c): #shot, weights, capacity
    ''' Takes Predicted value, Weights of shots and expected video length as input 
    and returns an array of chosen shot indices'''


    shot = len(s) + 1 #number of shots
    cap = c + 1 #capacity threshold

    #matching the modified size by adding 0 at 0th index
    s = np.r_[[0], s] #adding 0 at 0th index (concatinating)
    w = np.r_[[0], w] #adding 0 at 0th index (concatinating)

    #Creating and Filling Dynamic Programming Table with zeros in shot x cap dimensions
    dp = [] #creating empty list or table
    for j in range(shot):
        dp.append([]) #s+1 rows
        for i in range(cap):
            dp[j].append(0) #c+1 columns

    #Started filling values from (2nd row, 2nd column) till (shot X cap) and keeping the values for 0th indexes as 0
    #following dynamic programming approach to fill values
    for i in range(1,shot):
        for j in range(1,cap):
            if w[i] <= j:
                dp[i][j] = max(s[i] + dp[i-1][j-w[i]], dp[i-1][j])
            else:
                dp[i][j] = dp[i-1][j]

    #choosing the optimal pair of keyshots
    choice = []
    i = shot - 1
    j = cap - 1
    while i > 0 and j > 0:
        if dp[i][j] != dp[i-1][j]: #starting from last element and going further
            choice.append(i-1)
            j = j - w[i]
            i = i - 1
        else:
            i = i - 1

    return dp[shot-1][cap-1], choice

In [ ]:
def eval_metrics(y_pred, y_true):
  ''' Takes Predicted and True value as input and returns various evaluation metrics'''
    overlap = np.sum(y_pred * y_true)
    precision = overlap / (np.sum(y_pred) + 1e-8)
    recall = overlap / (np.sum(y_true) + 1e-8)
    if precision == 0 and recall == 0:
        fscore = 0
    else:
        fscore = 2 * precision * recall / (precision + recall)
    return [precision, recall, fscore]

In [ ]:
def select_keyshots(video_info, pred_score):
  '''Takes Video Info and predicted score array as input and returns predicted score,
  selected shots and predicted summary'''
    vidlen = video_info['length'][()]
    cps = video_info['change_points'][:]
    weight = video_info['n_frame_per_seg'][:]
    pred_score = np.array(pred_score)
    pred_score = upsample(pred_score, vidlen)
    pred_value = np.array([pred_score[cp[0]:cp[1]].mean() for cp in cps])
    _, selected = knapsack(pred_value, weight, int(0.15 * vidlen))
    selected = selected[::-1]
    key_labels = np.zeros((vidlen,))
    for i in selected:
        key_labels[cps[i][0]:cps[i][1]] = 1
    return pred_score.tolist(), selected, key_labels.tolist()

In [ ]:
def upsample(down_arr, vidlen):
    up_arr = np.zeros(vidlen)
    ratio = vidlen // 320
    l = (vidlen - ratio * 320) // 2
    i = 0
    while i < 320:
        up_arr[l:l+ratio] = np.ones(ratio, dtype = int) * down_arr[0][i]
        l += ratio
        i += 1
    return up_arr